In [3]:
import os
import json
import math
from pathlib import Path
from tqdm import tqdm
from torchvision.transforms.functional import to_pil_image
import sys
sys.path.append("..")

from data import PollenDataset
import torchvision.transforms as transforms
dataset = PollenDataset(image_transforms=transforms.ToTensor())

(left_img, right_img), (vertices, faces) = dataset[0]
    

# Zielverzeichnis für GS
output_root = "gaussian_splatting_input"
os.makedirs(output_root, exist_ok=True)

def create_transforms_json(scene_dir):
    """Erstellt eine transforms.json mit 2 orthogonalen Kamerapositionen."""
    transform_data = {
        "camera_angle_x": math.radians(50),  # ~50mm FOV
        "frames": [
            {
                "file_path": "images/view_000",
                "transform_matrix": [
                    [1, 0, 0, 0],
                    [0, 0, -1, 0],
                    [0, 1, 0, 1],
                    [0, 0, 0, 1]
                ]
            },
            {
                "file_path": "images/view_001",
                "transform_matrix": [
                    [1, 0, 0, 0],
                    [0, 1, 0, 1],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]
                ]
            }
        ]
    }

    out_path = os.path.join(scene_dir, "transforms.json")
    with open(out_path, "w") as f:
        json.dump(transform_data, f, indent=4)
    return out_path


def save_sample_for_gs(index, left_tensor, right_tensor):
    """Speichert Bilder und JSON für ein Sample"""
    scene_name = f"scene_{index:03d}"
    scene_dir = os.path.join(output_root, scene_name)
    image_dir = os.path.join(scene_dir, "images")
    Path(image_dir).mkdir(parents=True, exist_ok=True)

    # Speichern der Bilder
    left_img = to_pil_image(left_tensor)
    right_img = to_pil_image(right_tensor)
    left_img.save(os.path.join(image_dir, "view_000.png"))
    right_img.save(os.path.join(image_dir, "view_001.png"))

    # JSON generieren
    create_transforms_json(scene_dir)


# Main Loop: Alle Samples verarbeiten
for idx in tqdm(range(len(dataset)), desc="Exporting scenes for Gaussian Splatting"):
    (left, right), _ = dataset[idx]
    save_sample_for_gs(idx, left, right)

print(f"\n✅ Export complete. All scenes saved in: {output_root}")


Exporting scenes for Gaussian Splatting: 100%|██████████| 207/207 [00:10<00:00, 20.26it/s]


✅ Export complete. All scenes saved in: gaussian_splatting_input
